In [1]:
from xgboost import XGBClassifier
import warnings
from tabpfn import TabPFNClassifier
import numpy as np
import pandas as pd
import os
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from data_prep_utils import *
from evaluate import *
from load_models import *
import matplotlib.pyplot as plt
import torch
import openml

In [2]:
path = "datasets/data_all.csv"
data, labels = get_microbiome(path)
data = top_non_zero(data)
data, labels = unison_shuffled_copies(data, labels)

In [3]:
n_optim = 100
ft_epochs = 10
sampling = undersample
metrics = metrics = ["accuracy", "precision", "recall", "roc_auc"]
models = [
    XGBClassifier(n_estimators=5, max_depth=5, learning_rate=1, objective='binary:logistic'),
    XGBoostOptim(n_optim=n_optim),
    LogisticRegression(max_iter=500), 
    TabPFNClassifier(device='cpu', N_ensemble_configurations=3),
    TabForestPFNClassifier("saved_models/tabforest/mix600k/tabforestpfn.pt", "saved_models/tabforest/mix600k/config_run.yaml")
]
results = pd.DataFrame(np.zeros((len(models), len(metrics))), 
                       index=[m.__class__.__name__ for m in models],
                      columns=metrics)

for ii, model in enumerate(models):
    results.iloc[ii,:] = cross_validate_sample(model, data, labels, metrics)

[W 2024-07-04 13:48:11,283] Trial 96 failed with parameters: {'booster': 'gbtree', 'lambda': 0.012658061423251472, 'alpha': 0.0036406352257367016, 'subsample': 0.41013737773066594, 'colsample_bytree': 0.694277129388829, 'max_depth': 3, 'min_child_weight': 3, 'eta': 0.028869633325889647, 'gamma': 0.036030512093258166, 'grow_policy': 'depthwise'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\lenna\Desktop\MT\TabPFN-medical\load_models.py", line 116, in objective
    model.fit(X_train, y_train)
  File "C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\lenna\anaconda3\envs\master3\Lib\site-packages\xgboost\sklearn.py", line 1531, in fit
  

In [ ]:
print(results)

In [ ]:
data, labels = oversample(data, labels)